# Week6 - Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

Team Members:

Sai Chaitanya Varma Indukuri - PN45610

Hemanth Gorapalli - PC60253

Jayakrishna Konda - ZC39972

Surya Teja Indukuri - UI50216

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

In [2]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [3]:
df.head().T

,0,1,2,3,4
Loan_ID,LP001003,LP001005,LP001006,LP001008,LP001013
Gender,Male,Male,Male,Male,Male
Married,Yes,Yes,Yes,No,Yes
Dependents,1,0,0,0,0
Education,Graduate,Graduate,Not Graduate,Graduate,Not Graduate
Self_Employed,No,Yes,No,No,No
ApplicantIncome,4583,3000,2583,6000,2333
CoapplicantIncome,1508.0,0.0,2358.0,0.0,1516.0
LoanAmount,128.0,66.0,120.0,141.0,95.0
Loan_Amount_Term,360.0,360.0,360.0,360.0,360.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


In [5]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,381.000000,381.000000,381.000000,370.000000,351.000000
mean,3579.845144,1277.275381,104.986877,340.864865,0.837607
std,1419.813818,2340.818114,28.358464,68.549257,0.369338
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2600.000000,0.000000,90.000000,360.000000,1.000000
50%,3333.000000,983.000000,110.000000,360.000000,1.000000
75%,4288.000000,2016.000000,127.000000,360.000000,1.000000
max,9703.000000,33837.000000,150.000000,480.000000,1.000000


In [6]:
categorical_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area', 'Loan_Status']
for col in categorical_cols:
    print(col)
    print(df[col].value_counts())

numerical_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for col in numerical_cols:
    print(col)
    print(df[col].describe())

print("Duplicate Loan IDs:", df.duplicated('Loan_ID').sum())


Gender
Gender
Male      291
Female     85
Name: count, dtype: int64
Married
Married
Yes    228
No     153
Name: count, dtype: int64
Dependents
Dependents
0     234
2      59
1      52
3+     28
Name: count, dtype: int64
Education
Education
Graduate        278
Not Graduate    103
Name: count, dtype: int64
Self_Employed
Self_Employed
No     325
Yes     35
Name: count, dtype: int64
Credit_History
Credit_History
1.0    294
0.0     57
Name: count, dtype: int64
Property_Area
Property_Area
Semiurban    149
Urban        126
Rural        106
Name: count, dtype: int64
Loan_Status
Loan_Status
Y    271
N    110
Name: count, dtype: int64
ApplicantIncome
count     381.000000
mean     3579.845144
std      1419.813818
min       150.000000
25%      2600.000000
50%      3333.000000
75%      4288.000000
max      9703.000000
Name: ApplicantIncome, dtype: float64
CoapplicantIncome
count      381.000000
mean      1277.275381
std       2340.818114
min          0.000000
25%          0.000000
50%        983.00

In [7]:
print(df['Credit_History'].value_counts())
print(df['Dependents'].value_counts())

Credit_History
1.0    294
0.0     57
Name: count, dtype: int64
Dependents
0     234
2      59
1      52
3+     28
Name: count, dtype: int64


In [8]:
print(df.isnull().sum())

Loan_ID               0
Gender                5
Married               0
Dependents            8
Education             0
Self_Employed        21
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     11
Credit_History       30
Property_Area         0
Loan_Status           0
dtype: int64


In [9]:
print(df.dtypes)

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object


In [10]:
df.head().T

,0,1,2,3,4
Loan_ID,LP001003,LP001005,LP001006,LP001008,LP001013
Gender,Male,Male,Male,Male,Male
Married,Yes,Yes,Yes,No,Yes
Dependents,1,0,0,0,0
Education,Graduate,Graduate,Not Graduate,Graduate,Not Graduate
Self_Employed,No,Yes,No,No,No
ApplicantIncome,4583,3000,2583,6000,2333
CoapplicantIncome,1508.0,0.0,2358.0,0.0,1516.0
LoanAmount,128.0,66.0,120.0,141.0,95.0
Loan_Amount_Term,360.0,360.0,360.0,360.0,360.0


In [11]:
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
categorical_vars = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']

print("Numerical Variables:")
print("-" * 20)
for var in numerical_vars:
    print(f"{var}: Represents a numerical value, suitable for mathematical operations.")

print("\n")

print("Categorical Variables:")
print("-" * 20)
for var in categorical_vars:
    print(f"{var}: Represents a categorical variable with distinct categories.")


Numerical Variables:
--------------------
ApplicantIncome: Represents a numerical value, suitable for mathematical operations.
CoapplicantIncome: Represents a numerical value, suitable for mathematical operations.
LoanAmount: Represents a numerical value, suitable for mathematical operations.
Loan_Amount_Term: Represents a numerical value, suitable for mathematical operations.


Categorical Variables:
--------------------
Loan_ID: Represents a categorical variable with distinct categories.
Gender: Represents a categorical variable with distinct categories.
Married: Represents a categorical variable with distinct categories.
Dependents: Represents a categorical variable with distinct categories.
Education: Represents a categorical variable with distinct categories.
Self_Employed: Represents a categorical variable with distinct categories.
Credit_History: Represents a categorical variable with distinct categories.
Property_Area: Represents a categorical variable with distinct categories.

In [12]:
from sklearn.model_selection import train_test_split

X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=120, stratify=y)

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
imputer = SimpleImputer(strategy='most_frequent')

In [13]:
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Verify pipelines
X_train_transformed_numerical = numerical_pipeline.fit_transform(X_train[numerical_vars])
X_train_transformed_categorical = categorical_pipeline.fit_transform(X_train[categorical_vars])

print("\nShape of transformed numerical data:", X_train_transformed_numerical.shape)
print("Shape of transformed categorical data:", X_train_transformed_categorical.shape)


Shape of transformed numerical data: (304, 4)
Shape of transformed categorical data: (304, 321)


In [14]:
preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_vars),
    ('categorical', categorical_pipeline, categorical_vars)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

train_preds = pipeline.predict(X_train)
test_preds = pipeline.predict(X_test)

train_accuracy = accuracy_score(y_train, train_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print("\nAccuracy on training set:", train_accuracy)
print("Accuracy on test set:", test_accuracy)



Accuracy on training set: 0.8618421052631579
Accuracy on test set: 0.8441558441558441


Here we are adding the grid search and train and we compare the performance and suggest the model which will be the best

In [15]:
# Define the parameter grid
variable_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
}

grid_search = GridSearchCV(pipeline, param_grid = variable_grid, cv=5, scoring='accuracy',n_jobs = -1)

grid_search.fit(X_train, y_train)

best_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")

/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadul

Best parameters: {'classifier__C': 10, 'classifier__solver': 'newton-cg'}


/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadula/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/saiakhilsadul

In [16]:
train_accuracy = accuracy_score(y_train, best_estimator.predict(X_train))
test_accuracy = accuracy_score(y_test, best_estimator.predict(X_test))

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Train Accuracy: 1.0
Test Accuracy: 0.8441558441558441


We evaluate how well the model performs by using accuracy, as the measure. Accuracy is calculated by comparing the number of predictions, to the number of predictions made. This assessment is done individually for both the training and testing sets to ensure that the model is not overfitting to the training data. To improve the models accuracy we use grid search methods continuously tweaking its hyperparameters throughout the training phase.

When comparing the two models we assess their performance metrics specifically focusing on accuracy scores. The model that underwent training, without grid search shows a training accuracy of 0.8618421052631579 and a test accuracy of 0.8441558441558441. In contrast the model that was fine tuned using grid search exhibits a training accuracy of 1.0 and a test accuracy of 0.8441558441558441. Generally we prefer the model with test accuracy as it indicates performance, on new data. However it's important to ensure that the test accuracy doesn't significantly exceed the training accuracy to avoid overfitting concerns.